In [22]:
import pandas as pd
import numpy as np
from pathlib import Path

#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
data_folder = Path('E:/Downloads/')
#data_folder = Path('C:/Users/Paul/Downloads/')

df = pd.read_csv(data_folder/'train2_irf_0000_corr98.csv', \
                 dtype='int8', nrows=5000)
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8', nrows=5000)

print(df.shape, y.shape)

(5000, 18124) (5000, 1)


In [ ]:
%%time
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
import warnings
from sklearn.externals import joblib
from hyperopt.pyll.base import scope

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

def Performance(Model,Y,X):
    AUC = roc_auc_score(Y, Model.predict(X))
    aucs.append(AUC)
    print ('the AUC is : %0.4f' %  AUC)
    return AUC

def auc_model(params):
    clf = xgb.XGBClassifier(**params)
    clf.fit(train_x, train_y)
    #return cross_val_score(clf, train_x, train_y, scoring='roc_auc').mean()
    AUC = Performance(clf, test_y, test_x)
    return AUC

def last_model(params):
    clf = xgb.XGBClassifier(**params)
    clf.fit(s1_x, s1_y)
    AUC = Performance(clf, valid_y, valid_x)
    return clf, AUC

param_space = {
    'seed': 42,
    'n_estimators': scope.int(hp.uniform('n_estimators', 100, 500)),
    'eta': hp.quniform('eta', 0.025, 0.25, 0.025),
    'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'alpha'       : hp.uniform('alpha', 1e-4, 1e-6 ),
    'lambda'      : hp.uniform('lambda', 1e-4, 1e-6 ),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'eval_metric': 'auc',
    'objective': 'binary:logistic',
    'silent': 1
}
#'alpha' :  hp.quniform('alpha', 0, 10, 1),
#'lambda': hp.quniform('lambda', 1, 2, 0.1),
def f(params):
    global best
    global best_params
    auc = auc_model(params)
    if auc > best:
        best = auc
        best_params = params
        #print ('new best:', best, params)
    return {'loss': -auc, 'status': STATUS_OK}

aucs = []

s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20)
for t1_index, val_index in s1.split(df, y):
    s1_x, s1_y = pd.DataFrame(df.iloc[t1_index]), y.iloc[t1_index].values.ravel()
    valid_x, valid_y = df.iloc[val_index], y.iloc[val_index].values.ravel()
 
cv = StratifiedKFold(n_splits=4, shuffle=True)
for train_index, test_index in cv.split(s1_x, s1_y):
    train_x, train_y = pd.DataFrame(s1_x.iloc[train_index]), s1_y[train_index]
    test_x, test_y = pd.DataFrame(s1_x.iloc[test_index]), s1_y[test_index]

    trials = Trials()
    best = 0
    best = fmin(f, param_space, algo=tpe.suggest, max_evals=2, trials=trials)
    print ('\nbest:')
    model, AUC = last_model(best_params)
    output = 'XGB_' + str(int(AUC * 10000)) + '.pkl'
    #joblib.dump(model, data_folder/output)     
    print(space_eval(param_space, best))
    print('\n')
print(np.mean(aucs))

the AUC is : 0.6851
0.685133943381
the AUC is : 0.6805
0.680516112851

best:
the AUC is : 0.7179
{'alpha': 7.030579265435521e-05, 'colsample_bytree': 0.9500000000000001, 'eta': 0.05, 'eval_metric': 'auc', 'gamma': 0.6000000000000001, 'lambda': 3.0365664991660436e-05, 'max_depth': 3, 'min_child_weight': 6.0, 'n_estimators': 344, 'objective': 'binary:logistic', 'seed': 42, 'silent': 1, 'subsample': 0.6000000000000001}


the AUC is : 0.6724
0.672355211729
the AUC is : 0.6796
0.679625034464

best:


In [38]:
#0.? ENS
#0.7861 search 100
classifier = joblib.load(data_folder/'MLP_.pkl') 
#t1 = pd.read_csv(data_folder/'irf_weight_00010.csv', names=['a','b'])
test = pd.read_csv(data_folder/'test_redux.csv')
#t1.drop(['a'], axis=1, inplace=True)
#t1 = t1.drop(t1.index[:1])
#rfc_test = test.drop(t1['b'].tolist(), axis=1)
probas_ = classifier.predict_proba(rfc_test)[:, 1]
classifier.get_params()

the AUC is : 0.9991
the AUC is : 0.7644


In [ ]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = probas_
submission = submission[['Ids', 'TARGET']]
submission.to_csv(data_folder/'submission.csv', sep=';', index=False, header={'Ids', 'TARGET'})